In [14]:
# -----------------------> initialize libraries
import os
import json
import pandas as pd
from urllib.request import urlretrieve
from zipfile import ZipFile
import ijson
import numpy as np

# -----------------------> Download and extract FDA recall JSON
url = "https://download.open.fda.gov/device/recall/device-recall-0001-of-0001.json.zip"
zip_filename = "device-recall-0001-of-0001.json.zip"
json_filename = "device-recall-0001-of-0001.json"

if not os.path.exists(zip_filename):
    print("Downloading FDA recall zip...")
    urlretrieve(url, zip_filename)
else:
    print("ZIP already downloaded.")

if not os.path.exists(json_filename):
    print("Extracting JSON...")
    with ZipFile(zip_filename, 'r') as z:
        z.extract(json_filename)
else:
    print("JSON already extracted.")

# -----------------------> Parse recall JSON
records = []
with open(json_filename, 'r', encoding='utf-8') as f:
    parser = ijson.items(f, 'results.item')
    for entry in parser:
        manufacturer = entry.get('recalling_firm', '').strip() or "Unknown"

        recall_status = entry.get('recall_status', '').strip().lower()
        if 'terminated' in recall_status:
            rating = 1000
        elif 'completed' in recall_status:
            rating = 800
        elif 'ongoing' in recall_status:
            rating = 500
        elif 'pending' in recall_status:
            rating = 300
        else:
            rating = 600

        records.append({
            'manufacturer': manufacturer,
            'product_description': entry.get('product_description', '').replace('\n',' ').strip(),
            'recall_status': recall_status,
            'reason_for_recall': entry.get('reason_for_recall','').replace('\n',' ').strip(),
            'event_date_initiated': entry.get('event_date_initiated',''),
            'recall_number': entry.get('recall_number','').strip(),
            'rating': rating
        })

df = pd.DataFrame(records)
print(f"Parsed {len(df)} recalls")

# -----------------------> Download and extract FDA 510(k) JSON
k510_url = "https://download.open.fda.gov/device/510k/device-510k-0001-of-0001.json.zip"
k510_zip = "device-510k-0001-of-0001.json.zip"
k510_json = "device-510k-0001-of-0001.json"

if not os.path.exists(k510_zip):
    print("Downloading FDA 510(k) zip...")
    urlretrieve(k510_url, k510_zip)
else:
    print("510(k) ZIP already downloaded.")

if not os.path.exists(k510_json):
    print("Extracting 510(k) JSON...")
    with ZipFile(k510_zip, 'r') as z:
        z.extract(k510_json)
else:
    print("510(k) JSON already extracted.")

# -----------------------> Parse 510(k) JSON robustly
k510_records = []
with open(k510_json, 'r', encoding='utf-8') as f:
    parser = ijson.items(f, 'results.item')
    for entry in parser:
        manufacturer = (
            entry.get('applicant') or
            entry.get('applicant_name') or
            entry.get('manufacturer_name')
        )
        if not manufacturer:
            openfda = entry.get('openfda', {})
            manufacturer = openfda.get('manufacturer_name', [None])
            if isinstance(manufacturer, list):
                manufacturer = manufacturer[0]
        if not manufacturer:
            manufacturer = "Unknown"
        manufacturer = str(manufacturer).strip()
        k510_records.append({
            'manufacturer': manufacturer,
            'k_number': entry.get('k_number', '').strip(),
            'device_name': entry.get('device_name', '').strip(),
            'product_code': entry.get('product_code', '').strip(),
            'decision_date': entry.get('decision_date', '')
        })

k510_df = pd.DataFrame(k510_records)
k510_df['manufacturer'] = k510_df['manufacturer'].fillna('Unknown').astype(str).str.strip()

# Aggregate total 510(k) approvals per manufacturer
k510_summary = k510_df.groupby('manufacturer')['k_number'].nunique().reset_index(name='total_approvals')
print(f"Parsed {len(k510_summary)} unique 510(k) manufacturers")

# -----------------------> Clean recall dataframe
df['manufacturer'] = df['manufacturer'].fillna('Unknown').astype(str).str.strip()

# Compute total recalls per manufacturer
total_recalls = df.groupby('manufacturer')['recall_number'].nunique().reset_index(name='total_recalls')
df = pd.merge(df, total_recalls, on='manufacturer', how='left')

# Compute average rating per manufacturer (used for aggregation)
df['avg_rating'] = df.groupby('manufacturer')['rating'].transform('mean')

# Compute ELO
avg_rating_value = df['rating'].mean()
df['elo'] = 1000 - 0.04 * (df['rating'] - avg_rating_value)

# Short company code
df['Company_Short'] = df['manufacturer'].str.upper().str.slice(0, 4)

# -----------------------> Aggregate recall summary by Company_Short
agg_dict = {
    'manufacturer': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',
    'total_recalls': 'sum',
    'avg_rating': 'mean',
    'elo': 'mean'
}

summary_df = df.groupby('Company_Short', as_index=False).agg(agg_dict)

# -----------------------> Aggregate 510(k) approvals by Company_Short
k510_summary['Company_Short'] = k510_summary['manufacturer'].str.upper().str.slice(0, 4)
k510_agg = k510_summary.groupby('Company_Short', as_index=False).agg(
    total_approvals=('total_approvals', 'sum')
)

# -----------------------> Merge approvals into recall summary
merged_df = pd.merge(summary_df, k510_agg, on='Company_Short', how='left')
merged_df['total_approvals'] = merged_df['total_approvals'].fillna(0).astype(int)

# Compute normalized metric: recalls per device
merged_df['recalls_per_device'] = merged_df['total_recalls'] / merged_df['total_approvals'].replace(0, np.nan)
merged_df['recalls_per_device'] = merged_df['recalls_per_device'].fillna(0)

# -----------------------> Add rating category
percentiles = summary_df['elo'].rank(pct=True)
conditions = [
    percentiles >= 0.8,
    (percentiles >= 0.6) & (percentiles < 0.8),
    (percentiles >= 0.4) & (percentiles < 0.6),
    (percentiles >= 0.2) & (percentiles < 0.4),
    percentiles < 0.2
]
ratings = ['Excellent', 'Great', 'Average', 'Poor', 'Avoid']
summary_df['Rating_Category'] = np.select(conditions, ratings, default='Unrated').astype(str)

# -----------------------> Export Excel
output_file = "recalls_with_ratings_and_510k.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    merged_df.to_excel(writer, sheet_name="Summary", index=False)
    summary_df.to_excel(writer, sheet_name="Recalls_Only", index=False)
    k510_summary.to_excel(writer, sheet_name="Approvals_Only", index=False)

print(f"✅ Saved combined recall + 510(k) analysis to {output_file}")


ZIP already downloaded.
JSON already extracted.
Parsed 56268 recalls
510(k) ZIP already downloaded.
510(k) JSON already extracted.
Parsed 39429 unique 510(k) manufacturers


Exception ignored in: <generator object utf8_encoder at 0x0000022981B46350>
Traceback (most recent call last):
  File "C:\Users\Mitchell Esbenshade\anaconda3\Lib\site-packages\ijson\backends\python.py", line 46, in utf8_encoder
    target.close()
  File "C:\Users\Mitchell Esbenshade\anaconda3\Lib\site-packages\ijson\backends\python.py", line 116, in Lexer
    target.send(EOF)
  File "C:\Users\Mitchell Esbenshade\anaconda3\Lib\site-packages\ijson\backends\python.py", line 161, in parse_value
    raise common.IncompleteJSONError('Incomplete JSON content')
ijson.common.IncompleteJSONError: Incomplete JSON content


✅ Saved combined recall + 510(k) analysis to recalls_with_ratings_and_510k.xlsx
